# Ethereum Transaction Data Analysis with Python and PySpark

This notebook, "etherscanAnalysis.ipynb", is designed for analyzing Ethereum transaction data. It employs a combination of Python standard libraries and PySpark, a powerful tool for large-scale data processing. The primary steps in the notebook are:

1. **Initial Setup**: The notebook begins by importing necessary libraries and setting paths for input and output files containing Ethereum transaction data.
2. **Data Processing with Python**: It includes Python scripts for handling large CSV files, defining and retaining key transaction attributes such as addresses, timestamps, and values. The data is filtered based on these attributes and saved to a new CSV file. Additionally, it counts and prints the number of rows based on a specific condition in the dataset.
3. **Data Processing Using PySpark**: Leveraging PySpark's distributed computing capabilities, the notebook reads the CSV data into a Spark DataFrame, performs data transformations, and writes the processed data to a new file. The notebook also uses PySpark to count and filter rows based on certain conditions.

This notebook is an effective tool for extracting, filtering, and processing large volumes of Ethereum blockchain transaction data, showcasing the synergy between Python and PySpark for data analysis.


In [ ]:
import csv
import sys
input_file = '/mnt/c/Users/korchien/Downloads/server-3-152.7.177.227/ExternalTransactionItem.csv'
output_file = '/mnt/c/Users/korchien/Downloads/server-3-152.7.177.227/FilteredExternalTransactionItem.csv'

In [4]:
# Increase the maximum field size limit
max_int = sys.maxsize
while True:
    try:
        csv.field_size_limit(max_int)
        break
    except OverflowError:
        max_int = int(max_int / 2)

# Define the columns to keep
columns_to_keep = ['address_from', 'address_to', 'input', 'timestamp', 'transaction_hash', 'value']

with open(input_file, mode='r', encoding='utf-8') as infile, \
     open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    writer = csv.DictWriter(outfile, fieldnames=columns_to_keep)

    # Write the header based on the columns to keep
    writer.writeheader()

    count_0x = 0  # Initialize a counter for rows with '0x'

    for row in reader:
        # Check if 'input' column is not '0x'
        if row.get('input', '') != '0x':  
            # Write only the columns we want to keep
            writer.writerow({col: row[col] for col in columns_to_keep})
        else:
            count_0x += 1

print(f"Number of rows with '0x' in the specified column: {count_0x}")


In [ ]:
print(count_0x)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

# Initialize a SparkSession
spark = SparkSession.builder \
    .appName("Filter and Select Columns") \
    .getOrCreate()

In [ ]:
# Read the CSV file into a DataFrame
df = spark.read.csv(input_file, header=True, inferSchema=True)

# Select only the columns to keep
df = df.select(*columns_to_keep)

# Count the number of rows where 'input' column is '0x'
count_0x = df.filter(col('input') == '0x').count()

# Filter out rows where 'input' column is '0x'
df_filtered = df.filter(col('input') != '0x')

# Write the filtered DataFrame to a new CSV file
df_filtered.write.csv(output_file, header=True, mode='overwrite')

# Print the count of '0x' rows
print(f"Number of rows with '0x' in the specified column: {count_0x}")

In [ ]:
# Read the CSV file into a DataFrame
df = spark.read.csv(output_file, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

In [ ]:
# Count the number of rows in the DataFrame
row_count = df.count()

print(f"The DataFrame contains {row_count} rows.")